In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaTokenizer
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
class RoBERTaClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClass, self).__init__()
        self.roberta_model = RobertaModel.from_pretrained("roberta-base",return_dict = True)
        self.pre_classifier = torch.nn.Linear(768,768)
        self.dropout = torch.nn.Dropout(0.3)
        self.activation = torch.nn.Tanh()
        self.classifier = torch.nn.Linear(768, 14)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.roberta_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_pre_classifier = self.pre_classifier(output.pooler_output)
        output_dropout = self.dropout(output_pre_classifier)
        output_activation = self.activation(output_dropout)
        output = self.classifier(output_activation)
        return output

model = RoBERTaClass()
model.to(device)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base',truncation=True, do_lower_case=True)

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.text = df['Text']
        self.targets = self.df[labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        title = str(self.text[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
def output_labels(outputs, threshold=0.5):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(outputs)
  preds = np.zeros(probs.shape)
  preds[np.where(probs.cpu()>=threshold)] = 1
  return preds

def calculate_accuracy(preds, targets):

    labels = targets.cpu().numpy()
    row_comparison = np.equal(preds, labels)
    num_same_positions = np.sum(row_comparison, axis=1)
    n_correct = np.sum(num_same_positions == preds.shape[1])

    return n_correct

def metrics(predictions, labels):
    y_true = labels
    y_pred = predictions
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

    # Calculate micro average
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')

    # Calculate macro average
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

    # Print precision, recall, and F1-score for each class
    for i in range(len(precision)):
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1-Score: {f1[i]}")
        print()

    # Print aggregate metrics
    print("Aggregate Metrics:")
    print(f"  Micro Average:")
    print(f"    Precision: {micro_precision}")
    print(f"    Recall: {micro_recall}")
    print(f"    F1-Score: {micro_f1}")
    print()
    print(f"  Macro Average:")
    print(f"    Precision: {macro_precision}")
    print(f"    Recall: {macro_recall}")
    print(f"    F1-Score: {macro_f1}")

In [ ]:
def test_inference(test_data_loader,model):
    total_targets = []
    total_outputs = []
    model.eval()
    for batch_idx, data in (enumerate(test_data_loader)):
        with torch.no_grad():
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            total_targets.extend(targets.tolist())
            preds = output_labels(outputs,0.5)
            total_outputs.extend(preds)
          

In [ ]:
df_test = pd.read_csv('sample_dataframe.csv')
test_dataset = Dataset(df_test, tokenizer, 256)
test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0
)